In [2]:
import os
import random
import numpy as np
import pandas as pd
import json

In [3]:
#create a dataframe containing the names of all of the jpgs in the full dataset
fileList = []
for file in os.listdir('C:/Users/miapi/Downloads/ttpla_dataset/sized_data'):
    if file.endswith(".jpg"):
        fileList.append(file)
        
df = pd.DataFrame(fileList)

#also creating an index to use later in extracting the img labels from the json files
index = df.index

#formatting the dataframe
df = df.rename(columns={0: "Id"}) 
df['labels'] = df.Id
df['labels'] = df['labels'].str.replace('.jpg','')
df['tower_wooden'] = ""
df['tower_lattice'] = ""
df['tower_tucohy'] = ""

df.head()

,Id,labels,tower_wooden,tower_lattice,tower_tucohy
0,04_2220.jpg,04_2220,,,
1,04_3420.jpg,04_3420,,,
2,04_585.jpg,04_585,,,
3,06_5355.jpg,06_5355,,,
4,07_1035.jpg,07_1035,,,


In [4]:
index = df.index
for file in os.listdir('C:/Users/miapi/Downloads/ttpla_dataset/sized_data'):
    if file.endswith(".json"):
        
        #load file as json object
        filePath = ("C:/Users/miapi/Downloads/ttpla_dataset/sized_data/"+file)
        f = open(filePath,)
        json_file = json.load(f)
        #get file namme without extension
        file2 = file.replace('.json','')
        
        #find designated jpeg (row in df)
        #rslt_df = df.loc[df['labels'] == file2]
        apples_indices = index[df['labels'] == file2]
        
        #parse json file to see if tower_wooden, tower_lattice, or tower_tucohy exists
        #add 0 or 1 to the corresponding dataframe row accordingly
        
        for element in json_file['shapes']:
            
            #tower wooden
            bool_wood = any(sd['label']=='tower_wooden' for sd in json_file['shapes'])  
            if bool_wood:
                df.at[apples_indices,'tower_wooden']= 1
            else:
                df.at[apples_indices,'tower_wooden']= 0 
                
            #tower lattice
            bool_lat = any(sd['label']=='tower_lattice' for sd in json_file['shapes']) 
            if bool_lat:
                df.at[apples_indices,'tower_lattice']= 1
            else:
                df.at[apples_indices,'tower_lattice']= 0
                
            #tower tucohy
            bool_tuc = any(sd['label']=='tower_tucohy' for sd in json_file['shapes']) 
            if bool_tuc:
                df.at[apples_indices,'tower_tucohy']= 1
            else:
                df.at[apples_indices,'tower_tucohy']= 0

In [ ]:
#show the whole dataframe
pd.set_option('display.max_rows', 1250)
df

In [ ]:
#checking to see if the labels were coded properly
df.tower_wooden.unique()
blankDat = df.loc[df['tower_wooden'] == '']
blankDat.head()

In [5]:
#create column containing info for all of the towers
df['towers'] = df[['tower_wooden', 'tower_lattice','tower_tucohy']].values.tolist()
df['tower_labels'] = np.empty((len(df), 0)).tolist()
for i in range(len(df)):
    
    if df['towers'][i][0] == 1:
        df['tower_labels'][i].append("tower_wooden")
    
    if df['towers'][i][1] == 1:
        df['tower_labels'][i].append("tower_lattice")
        
    if df['towers'][i][2] == 1:
        df['tower_labels'][i].append("tower_tucohy")
df.head()

,Id,labels,tower_wooden,tower_lattice,tower_tucohy,towers,tower_labels
0,04_2220.jpg,04_2220,0,0,0,"[0, 0, 0]",[]
1,04_3420.jpg,04_3420,1,0,0,"[1, 0, 0]",[tower_wooden]
2,04_585.jpg,04_585,1,0,0,"[1, 0, 0]",[tower_wooden]
3,06_5355.jpg,06_5355,0,0,0,"[0, 0, 0]",[]
4,07_1035.jpg,07_1035,1,1,0,"[1, 1, 0]","[tower_wooden, tower_lattice]"


In [ ]:
if (df['towers'][1][0] == 1):
    print('pizza')

In [ ]:
df['towers'][1][0]

In [6]:
#need to split the data into testing, training, and validating
random.seed(123)
train, validate, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])

In [7]:
train.head()

,Id,labels,tower_wooden,tower_lattice,tower_tucohy,towers,tower_labels
916,57_00986.jpg,57_00986,1,0,0,"[1, 0, 0]",[tower_wooden]
86,13_00184.jpg,13_00184,0,1,0,"[0, 1, 0]",[tower_lattice]
247,25_00356.jpg,25_00356,0,0,1,"[0, 0, 1]",[tower_tucohy]
355,31_00832.jpg,31_00832,0,0,0,"[0, 0, 0]",[]
618,42_00561.jpg,42_00561,0,0,0,"[0, 0, 0]",[]


In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline

In [ ]:
train.iloc[1][0]

In [ ]:
validate.iloc[1][0]

In [ ]:
test.iloc[1][0]

In [9]:
#reading in all of the training images
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('C:/Users/miapi/Downloads/ttpla_dataset/sized_data/'+train.iloc[i][0],target_size=(400,400,3))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|███████████████████████████████████████████████████████████████████████████████| 744/744 [00:06<00:00, 108.30it/s]


In [10]:
#check shape of X
X.shape

(744, 400, 400, 3)

In [ ]:
#visualizing one image 
plt.imshow(X[2])

In [ ]:
train['tower_labels'][2]

In [11]:
#validate
#reading in all of the training images
valid_image = []
for i in tqdm(range(validate.shape[0])):
    img = image.load_img('C:/Users/miapi/Downloads/ttpla_dataset/sized_data/'+validate.iloc[i][0],target_size=(400,400,3))
    img = image.img_to_array(img)
    img = img/255
    valid_image.append(img)
X_valid = np.array(valid_image)

100%|████████████████████████████████████████████████████████████████████████████████| 248/248 [00:02<00:00, 99.77it/s]


In [12]:
X_valid.shape

(248, 400, 400, 3)

In [13]:
#test
#reading in all of the training images
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('C:/Users/miapi/Downloads/ttpla_dataset/sized_data/'+test.iloc[i][0],target_size=(400,400,3))
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
X_test = np.array(test_image)

100%|███████████████████████████████████████████████████████████████████████████████| 249/249 [00:02<00:00, 103.66it/s]


In [14]:
X_test.shape

(249, 400, 400, 3)

In [15]:
#remove the label columns from  train file 
#convert the remaining columns to an array which will be the target for our images:

#need to df['DataFrame Column'] = df['DataFrame Column'].astype(float)first convert the column values from strings to floats
df['tower_wooden'] = df['tower_wooden'].astype(int)
df['tower_lattice'] = df['tower_lattice'].astype(int)
df['tower_tucohy'] = df['tower_tucohy'].astype(int)
y = np.array(train.drop(['Id', 'labels','towers','tower_labels'],axis=1))
print(y.shape)
y

(744, 3)


array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=object)

In [16]:
#validation set y
y_validate = np.array(validate.drop(['Id', 'labels','towers','tower_labels'],axis=1))
print(y_validate.shape)


(248, 3)


In [17]:
#test set y
y_test = np.array(test.drop(['Id', 'labels','towers','tower_labels'],axis=1))
print(y_test.shape)


(249, 3)


In [ ]:
#model architecture
#modify by changing the number of hidden layers, activation functions and other hyperparameters
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(400,400,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
#can use other optimizers/losses
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
X_train=np.asarray(X).astype(np.int)
y_train=np.asarray(y).astype(np.int)
X_test=np.asarray(X_valid).astype(np.int)
y_test=np.asarray(y_validate).astype(np.int)

In [ ]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=64)

## Trying different architecture

In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

In [19]:
X_train=np.asarray(X).astype(np.int)
y_train=np.asarray(y).astype(np.int)
X_valid=np.asarray(X_valid).astype(np.int)
y_valid=np.asarray(y_validate).astype(np.int)
X_test=np.asarray(X_test).astype(np.int)
y_test=np.asarray(y_test).astype(np.int)

## Alex Net for multilabel classification
takes extremely long (over 2 hours) to finish, not good accuracy

potential solutions: need to reformat X and y

In [20]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(400,400,3)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(3, activation='sigmoid'))
# compile model
opt = keras.optimizers.SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1.0/255.0)

In [22]:
train_it = datagen.flow(X_train, y_train, batch_size=16)
test_it = datagen.flow(X_valid, y_valid, batch_size=16)

In [23]:
history = model.fit(train_it, steps_per_epoch=len(train_it),
validation_data=test_it, validation_steps=len(test_it), epochs=50, verbose=0)

C:\Users\miapi\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


KeyboardInterrupt: 

In [25]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), batch_size=64)

Epoch 1/20
12/12 [==============================] - 627s 52s/step - loss: 0.4384 - accuracy: 0.4476 - val_loss: 0.4796 - val_accuracy: 0.4516
Epoch 2/20
12/12 [==============================] - 621s 52s/step - loss: 0.4160 - accuracy: 0.4946 - val_loss: 0.4857 - val_accuracy: 0.4234
Epoch 3/20
12/12 [==============================] - 606s 50s/step - loss: 0.3851 - accuracy: 0.4960 - val_loss: 0.4697 - val_accuracy: 0.4274
Epoch 4/20
12/12 [==============================] - 645s 54s/step - loss: 0.3536 - accuracy: 0.4651 - val_loss: 0.4874 - val_accuracy: 0.4758
Epoch 5/20
12/12 [==============================] - 650s 54s/step - loss: 0.3076 - accuracy: 0.5108 - val_loss: 0.5262 - val_accuracy: 0.4234
Epoch 6/20
12/12 [==============================] - 607s 51s/step - loss: 0.2562 - accuracy: 0.4906 - val_loss: 0.6787 - val_accuracy: 0.4960
Epoch 7/20
12/12 [==============================] - 642s 54s/step - loss: 0.2079 - accuracy: 0.5108 - val_loss: 0.8925 - val_accuracy: 0.4516
Epoch 

KeyboardInterrupt: 